In [1]:
%reset
#%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from auxiliary import AuxFuncs
from IPython.utils import io

gc.collect()

0

In [3]:
epochs_dir = 'C:\AnatArzData\YKM_data\epochs_and_evoked_allSubs'
prepro_name = "referenced"
import_type = "5Electorodes_plainEEGLAB"
trial_exclution_str = "_excOulierTrials-3.5" #"_excOulierTrials-2.5"

## Import epochs

In [ ]:
import_path = f'{epochs_dir}\\{import_type}{trial_exclution_str}.pkl'
aux = AuxFuncs(import_path)

allEpochs_perCond = aux.allEpochs_condIdDict
allEvents_df = aux.allEvents_df
c = aux.config
times = c['times']
time0_i = c['time0_i']

fig_output_dir = f"{c['outputs_dir_path']}/timefreq_clusterPerm"
if not os.path.exists(fig_output_dir):
    os.mkdir(fig_output_dir)

In [ ]:
def applyDesign(ax,title=''):
    ax.get_figure().patch.set_facecolor('xkcd:white')
    ax.set_facecolor('silver')
    ax.set_title(title)
    ax.legend(loc = 'upper right',prop={'size': 10})
    ax.axvline(x=0,color='gray', linestyle='--',label ="_nolegend_")
    ax.axhline(y=0, color='gray', linestyle='-',label ="_nolegend_")
    ax.set_ylabel('magnitude')
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_xlabel('Time (ms)')

## Time-freq functions

In [ ]:
cond_1 = {'TOA_cond':'Rand','Vigilance':'N3'}
cond_2 = {'TOA_cond':'Fixed','Vigilance':'N3'}
contrast_name = "allSubs_RandvsFixed_N3"

#### Parameters:
# ==================
# zscore baseline
## Tail is 0, so the statistic is thresholded on both sides of the distribution.
baseline_period = (None, 0) ## For timefreq analysis
p_value = 0.05 # default 0.05 # for clusters
decim = 1# default 2 for testing.. For reals- 1
n_permutations=1000 # default 1k for testing. For reals - 10K
min_freq = 4
freqs = np.arange(min_freq,80, 1)  # define frequencies of interest
n_cycles = freqs / min_freq # different number of cycle per frequency

tail = 0 # 0 = two-tailed test (for clusters (as we want positive and negative clusters))
p_value_pixels = 0.05

test_per_voxel='ttest'
is_tfr_performAverageBaseline=True

In [ ]:
def plot_clusters_map(ax1,T_obs,ch_idx,tfr_epochs,t_thresh,T_obs_plot):
    vmax = np.max(np.abs(T_obs))
    vmin = -vmax
    ax1.imshow(T_obs[ch_idx], cmap=plt.cm.gray,
               extent=[times[0], times[-1], freqs[0], freqs[-1]],
               aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
    ax1.imshow(T_obs_plot[ch_idx], cmap=plt.cm.RdBu_r,
               extent=[times[0], times[-1], freqs[0], freqs[-1]],
               aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
    #ax1.set_colorbar()
    ax1.set_xlabel('Time (ms)')
    ax1.set_ylabel('Frequency (Hz)')
    ax1.set_title(f'Induced power ({tfr_epochs.ch_names[ch_idx]})\nThreshold:{t_thresh}\ncluster p_val={p_value},n_permutation={n_permutations}\ncontrast:{cond_1};{cond_2}')
def plot_elecds_erps(ax2,mean_epochs_time_diff,ch_idx,tfr_epochs):
    ax2.plot(times,mean_epochs_time_diff.T,color='blue')
    mean_electrods = np.nanmean(mean_epochs_time_diff,axis=0)
    ax2.plot(times,mean_electrods,color='yellow',label='mean')
    ax2.plot(times,mean_epochs_time_diff[ch_idx,:],label=tfr_epochs.ch_names[ch_idx],color='red')
    ax2.legend()
    ax2.set_xlim(times[0],times[-1])
    applyDesign(ax2,'ERP difference')

## Run many contrasts at once

In [ ]:
def stat_fun_wilcox(X):
    result = scipy.stats.wilcoxon(X)
    return result.statistic

def getClustersPerConditions_5timesBaseline(c,cond_1,cond_2, contrast_name,cluster_range_index,is_tfr_performAverageBaseline=True,baseline_period=(None, 0),test_per_voxel='ttest'):
    if test_per_voxel=='ttest':
        test_per_voxel = mne.stats.ttest_1samp_no_p
    else:
        test_per_voxel = stat_fun_wilcox

    epochs_time_diff = []
    epochs_power_diff = []
    for sub in c['subs']:
        currContr_conds1 = cond_1.copy()
        currContr_conds1['Subject'] = sub
        __, cont_epochs1 = aux.getEvokedPerCondAndElectd(currContr_conds1,allEvents_df,allEpochs_perCond,times, outputType='mne')

        if cond_2 == "baseline":
            baseline_epochs_data = copy.deepcopy(cont_epochs1.get_data())

            baseline_orig_data = baseline_epochs_data[:,:,0:time0_i]
            baseline_epochs_data[:,:,time0_i:time0_i*2] = baseline_orig_data
            baseline_epochs_data[:,:,time0_i*2:time0_i*3] = baseline_orig_data
            baseline_epochs_data[:,:,time0_i*3:time0_i*4] = baseline_orig_data
            baseline_epochs_data[:,:,time0_i*4:time0_i*5] = baseline_orig_data
            baseline_epochs_data[:,:,time0_i*5:] = baseline_orig_data[:,:,:13]

            with io.capture_output() as captured: # suppress output
                cont_epochs2 = mne.EpochsArray(baseline_epochs_data, aux.info,tmin=-0.1)
        else:
            currContr_conds2 = cond_2.copy()
            currContr_conds2['Subject'] = sub
            __, cont_epochs2 = aux.getEvokedPerCondAndElectd(currContr_conds2,allEvents_df,allEpochs_perCond,times, outputType='mne')

        if is_tfr_performAverageBaseline:
            with io.capture_output() as captured: # suppress output
                tfr_epochs1 = tfr_morlet(cont_epochs1, freqs, n_cycles=n_cycles, decim=decim, average=False, return_itc=False)
                tfr_epochs2 = tfr_morlet(cont_epochs2, freqs, n_cycles=n_cycles, decim=decim, average=False, return_itc=False)

            both_tfr_data = np.concatenate((tfr_epochs1.data,tfr_epochs2.data),axis=0)
            both_tfr_data_tfr = copy.deepcopy(tfr_epochs1)
            both_tfr_data_tfr.data = both_tfr_data

            #Correction is applied to all epoch and channel together in the following way: 1.Calculate the mean signal of the baseline period. 2.Subtract this mean from the entire epoch.
            with io.capture_output() as captured: # suppress output
                both_tfr_data_tfr.apply_baseline(mode='zscore', baseline=baseline_period)

            num_of_cond1_trials = tfr_epochs1.data.shape[0]
            epochs_power1 = np.mean(both_tfr_data_tfr.data[:num_of_cond1_trials,:,:,:],axis=(0,1)) # elec, freqs, time
            epochs_power2 = np.mean(both_tfr_data_tfr.data[num_of_cond1_trials:,:,:,:],axis=(0,1)) # elec, freqs, time

            epochs_time_diff.append(np.mean(cont_epochs1,axis=0) - np.mean(cont_epochs2,axis=0))
        else: # apply baseline seperately
            tfr_epochs1 = tfr_morlet(cont_epochs1, freqs, n_cycles=n_cycles, decim=decim, average=True, return_itc=False)
            tfr_epochs2 = tfr_morlet(cont_epochs2, freqs, n_cycles=n_cycles, decim=decim, average=True, return_itc=False)
            tfr_epochs1.apply_baseline(mode='zscore', baseline=baseline_period)
            tfr_epochs2.apply_baseline(mode='zscore', baseline=baseline_period)
            epochs_power1 = tfr_epochs1.data # elec, freqs, time
            epochs_power2 = tfr_epochs2.data # elec, freqs, time

            epochs_time_diff.append(np.mean(cont_epochs1.get_data(),axis=0) - np.mean(cont_epochs2.get_data(),axis=0))

        means_diff = epochs_power1-epochs_power2
        epochs_power_diff.append(means_diff)

    # (n_epochs, n_channels, n_freqs, n_times)
    epochs_power_diff_arr = np.zeros((len(c['subs']),len(c['electrodes']),len(freqs),len(times)))
    for s,subject in enumerate(c['subs']):
        epochs_power_diff_arr[s,:,:,:] = epochs_power_diff[s]

    #### Define adjacency for statistics
    tfr_epochs = tfr_epochs1
    sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(tfr_epochs.info,ch_type=None)
    use_idx = [ch_names.index(ch_name) for ch_name in tfr_epochs.ch_names]
    sensor_adjacency = sensor_adjacency[use_idx][:, use_idx]
    assert sensor_adjacency.shape == (len(tfr_epochs.ch_names), len(tfr_epochs.ch_names))
    assert epochs_power_diff_arr.shape == (len(c['subs']), len(tfr_epochs.ch_names), len(tfr_epochs.freqs), len(tfr_epochs.times))
    adjacency = mne.stats.combine_adjacency(sensor_adjacency, len(tfr_epochs.freqs), len(tfr_epochs.times))
    assert adjacency.shape[0] == adjacency.shape[1] == len(tfr_epochs.ch_names) * len(tfr_epochs.freqs) * len(tfr_epochs.times)

    ### run cluster permutation
    degrees_of_freedom = len(c['subs']) - 1
    t_thresh = scipy.stats.t.ppf(1 - p_value_pixels / 2, df=degrees_of_freedom)
    T_obs, clusters, cluster_p_values, H0 = permutation_cluster_1samp_test(epochs_power_diff_arr[...,cluster_range_index[0]:cluster_range_index[1]], n_permutations=n_permutations, threshold=t_thresh, tail=tail,  out_type='mask', verbose=True, stat_fun=test_per_voxel)

    ############# plot
    T_obs_plot = np.nan * np.ones_like(T_obs)
    for c, clust_p_val in zip(clusters, cluster_p_values):
        if clust_p_val <= p_value:
            T_obs_plot[c] = T_obs[c]
    mean_epochs_time_diff = np.nanmean(epochs_time_diff,axis=0)

    padded_t_obs = np.zeros((len(c['electrodes']), len(freqs),len(times)))
    padded_t_obs_plot = np.zeros((len(c['electrodes']), len(freqs),len(times)))
    padded_t_obs[:,:,cluster_range_index[0]:cluster_range_index[1]]=T_obs
    padded_t_obs_plot[:,:,cluster_range_index[0]:cluster_range_index[1]]=T_obs_plot

    for ch_idx, elecd in enumerate(c['electrodes']):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        fig.subplots_adjust(left=0.12,bottom= 0.08,right= 0.96, top=0.85, wspace=0.2, hspace=0.3)
        plot_clusters_map(ax1,padded_t_obs,ch_idx,tfr_epochs,t_thresh,padded_t_obs_plot)
        plot_elecds_erps(ax2,mean_epochs_time_diff,ch_idx,tfr_epochs)
        plt.ioff()
        plt.savefig(f'{fig_output_dir}/timeFreq_clusters_{contrast_name}_{tfr_epochs.ch_names[ch_idx]}.png',bbox_inches='tight')
       
       
contrasts_5timesBase = {}    
contrasts_5timesBase["allSubs_RandvsFixed_Wn"] = {'cond_1':{'TOA_cond':'Rand','Vigilance':'Wn'},'cond_2':{'TOA_cond':'Fixed','Vigilance':'Wn'}}
for contrast in contrasts_5timesBase:
    cond_1 = contrasts_5timesBase[contrast]['cond_1']
    cond_2 = contrasts_5timesBase[contrast]['cond_2']
    print(f'Now computeing {cond_1} and {cond_2}')

    contrast_name = f"{contrast.title()}_-{test_per_voxel}"
    getClustersPerConditions_5timesBaseline(c,cond_1, cond_2, contrast_name,cluster_range_index=[time0_i, time0_i*2],is_tfr_performAverageBaseline=is_tfr_performAverageBaseline, test_per_voxel=test_per_voxel) 

: 

In [ ]:
np.zeros((len(c['electrodes']), len(freqs),len(times)))

: 